In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
plt.style.use("seaborn-colorblind")
%matplotlib inline

In [40]:
used_features = ["Temp", "PH","D.O. (mg/l)"]
water = pd.read_csv('waterdata.csv', usecols = used_features, encoding= 'unicode_escape')
# m = pd.read_csv('waterdata.csv', encoding= 'unicode_escape')
# target["D.O. (mg/l)"] = m["D.O. (mg/l)"]
print(water.shape)
water.head()
# target.head()

(1991, 3)


,Temp,D.O. (mg/l),PH
0,30.6,6.7,7.5
1,29.8,5.7,7.2
2,29.5,6.3,6.9
3,29.7,5.8,6.9
4,29.5,5.8,7.3


In [42]:
water["Temp"] = pd.to_numeric(water['Temp'], errors='coerce')
water["Temp"] = water["Temp"].replace(np.nan, 0)
water["PH"] = pd.to_numeric(water['PH'], errors='coerce')
water["PH"] = water["PH"].replace(np.nan, 0)
water["D.O. (mg/l)"] = pd.to_numeric(water["D.O. (mg/l)"], errors='coerce')
water["D.O. (mg/l)"] = water["D.O. (mg/l)"].replace(np.nan, 0)

In [43]:
water=water.mask(water["Temp"]==0).fillna(water["Temp"].mean())
water=water.mask(water["PH"]==0).fillna(water["PH"].mean())
water=water.mask(water["D.O. (mg/l)"]==0).fillna(water["D.O. (mg/l)"].mean())

In [44]:
water

,Temp,D.O. (mg/l),PH
0,30.600000,6.700000,7.500000
1,29.800000,5.700000,7.200000
2,29.500000,6.300000,6.900000
3,29.700000,5.800000,6.900000
4,29.500000,5.800000,7.300000
...,...,...,...
1986,24.998713,24.998713,24.998713
1987,29.000000,7.500000,585.000000
1988,28.000000,7.600000,98.000000
1989,28.000000,7.700000,91.000000


In [46]:
target = water["D.O. (mg/l)"]
features = water.drop('D.O. (mg/l)',axis=1)

In [47]:
features

,Temp,PH
0,30.600000,7.500000
1,29.800000,7.200000
2,29.500000,6.900000
3,29.700000,6.900000
4,29.500000,7.300000
...,...,...
1986,24.998713,24.998713
1987,29.000000,585.000000
1988,28.000000,98.000000
1989,28.000000,91.000000


In [48]:
target

0        6.700000
1        5.700000
2        6.300000
3        5.800000
4        5.800000
          ...    
1986    24.998713
1987     7.500000
1988     7.600000
1989     7.700000
1990     7.600000
Name: D.O. (mg/l), Length: 1991, dtype: float64

In [121]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
     water, target, test_size=0.33, random_state=42)

In [122]:
# numeric_columns = ["Temp", "PH","D.O. (mg/l)"]
numeric_columns = ["Temp", "PH"]
X_train.drop('D.O. (mg/l)',axis=1, inplace=True)
X_test.drop('D.O. (mg/l)',axis=1, inplace=True)
X_test

,Temp,PH
887,28.8,7.50
1670,16.0,7.00
414,28.0,8.20
1080,26.4,7.70
1102,28.1,6.90
...,...,...
1857,18.0,3.05
522,28.0,6.60
513,30.0,6.80
81,26.0,7.50


In [123]:
numeric_features = [tf.feature_column.numeric_column(key = column) for column in numeric_columns]
print(numeric_features[0])

NumericColumn(key='Temp', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [124]:
linear_features = numeric_features

In [125]:
training_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=32, shuffle=True, num_epochs=None)

In [126]:
eval_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_test, y=y_test, batch_size=32, shuffle=False, num_epochs = 1)

In [127]:
linear_regressor = tf.estimator.LinearRegressor(feature_columns=linear_features,
                                                model_dir = "linear_regressor")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'linear_regressor', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [128]:
linear_regressor.train(input_fn = training_input_fn,steps=2000)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_regressor/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 4000...
INFO:tensorflow:Saving checkpoints for 4000 into linear_regressor/model.ckpt.
INFO:tensorflow:Calling checkpoint liste

In [129]:
linear_regressor.evaluate(input_fn = eval_input_fn)

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-06T15:52:22Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_regressor/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.26107s
INFO:tensorflow:Finished evaluation at 2020-09-06-15:52:22
INFO:tensorflow:Saving dict for global step 6000: average_loss = 15.832931, glo

{'average_loss': 15.832931,
 'label/mean': 7.1882124,
 'loss': 16.302292,
 'prediction/mean': 7.082533,
 'global_step': 6000}

In [130]:
pred = list(linear_regressor.predict(input_fn = eval_input_fn))
pred = [p['predictions'][0] for p in pred]

INFO:tensorflow:Calling model_fn.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_regressor/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [131]:
prices = (pred)
print(prices)

[7.7698073, 4.650268, 7.57485, 7.1848993, 7.5991964, 6.28316, 7.8185596, 7.3311253, 4.650266, 7.5748096, 5.79573, 5.771349, 7.836882, 7.574816, 6.8437138, 5.576372, 7.5992002, 7.2823977, 7.5748367, 6.5999913, 7.3311195, 5.871384, 8.062252, 6.112559, 7.3311176, 6.112551, 7.745438, 8.06226, 7.4529953, 5.698229, 7.8185368, 7.721074, 7.0874004, 7.609673, 7.5748253, 5.381424, 7.452965, 7.3311195, 7.5748196, 7.818533, 7.0630426, 6.11247, 7.3311253, 5.1376963, 6.5756083, 8.062254, 7.745442, 7.8429155, 6.868068, 8.257228, 8.086633, 7.404247, 7.8185444, 7.355491, 7.5504537, 6.5999665, 7.33111, 7.4529624, 6.8436913, 7.745438, 7.5748143, 6.868074, 7.5017056, 7.5748215, 2.5098643, 8.013524, 7.5748386, 7.842925, 7.0873857, 3.4317036, 7.3554893, 6.356261, 7.940405, 6.8437138, 7.3067408, 8.110996, 7.404226, 7.5748386, 6.3562665, 7.3311214, 5.625142, 5.0402074, 7.69668, 5.1377, 7.8185463, 7.818533, 6.8437138, 5.8688455, 7.331108, 7.6479263, 7.0873985, 8.281595, 7.8267574, 7.332115, 7.5188904, 7.818546

In [132]:
X_test

,Temp,PH
887,28.8,7.50
1670,16.0,7.00
414,28.0,8.20
1080,26.4,7.70
1102,28.1,6.90
...,...,...
1857,18.0,3.05
522,28.0,6.60
513,30.0,6.80
81,26.0,7.50


In [133]:
y_test

887     6.5
1670    8.0
414     8.0
1080    6.8
1102    6.7
       ... 
1857    6.4
522     6.8
513     6.1
81      6.7
720     8.9
Name: D.O. (mg/l), Length: 658, dtype: float64

In [157]:
predict_x = {
    'Temp': [30.1],
    'PH': [7.5],
}

In [158]:
def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(10)

In [159]:
pred = linear_regressor.predict(
    input_fn=lambda: input_fn(predict_x))

In [160]:
pred

<generator object Estimator.predict at 0x7ff5e224fbd0>

In [161]:
pred = [p['predictions'][0] for p in pred]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from linear_regressor/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [162]:
pred

[8.086635]